often, it is hard to work on data which is very large and it just makes it easier to work with partitioned data.

OR in layman terms, devide and conquer

In [0]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import year, month, dayofmonth
#only import the methods that are needed while writing production code

class LoadAndPartition:
    src_path = '/mnt/testdata/green_tripdata_2021-01.csv'
    dst_path = '/mnt/testdata/Partition'
    
    def __init__(self):
        self.MySparkSession = SparkSession.builder.master('local').appName('partitionInADLSg2').getOrCreate()
    
    def extractDataSet(self) -> DataFrame:
        return self.MySparkSession.read.options(header=True,inferSchema=True).csv(self.src_path)
    
    def preProcess(self,df: DataFrame) -> DataFrame:
        return df.withColumn('Year',year('lpep_pickup_datetime')).withColumn('Month',month('lpep_pickup_datetime')).withColumn('Day',dayofmonth('lpep_pickup_datetime'))
    
    def partitionLoad(self, df: DataFrame):
        df.write.partitionBy('Year','Month','Day').mode('overwrite').save(self.dst_path)
        
    def run(self):
        self.partitionLoad(self.preProcess(self.extractDataSet()))

In [0]:
job1 =LoadAndPartition()
job1.run()